In [ ]:
# Get CSV files list from a folder
csv_files = glob.glob(DATA_PATH + "/*")
# load the dataframes and concatenate them
raw_df = pd.concat(map(pd.read_csv, csv_files))
raw_df = raw_df.set_index('Unnamed: 0').reset_index(drop=True)
raw_df.shape

In [ ]:
import pandas as pd
test = pd.read_csv('../Data/Processed/0_DataCleaned_df.csv')
test.head()

In [ ]:
import numpy as np

In [ ]:
test['company_name'].value_counts()

Meta                         17
DSMH LLC                      9
PayPal                        9
Disney                        7
Notion                        7
                             ..
Invisible Technologies        1
BJ Services                   1
Cybotic System                1
Infosys                       1
Toyota Research Institute     1
Name: company_name, Length: 874, dtype: int64

In [ ]:
test['comp_count'] = test.groupby('company_name')['company_name'].transform('count')

In [ ]:
test['companyname'] = np.where(test['comp_count'] >= 3, test['company_name'], np.nan)

In [ ]:
test['statecount'] = test.groupby('job_state')['job_state'].transform('count')

In [ ]:
test['jobstate'] = np.where(test['statecount'] >= 3, test['job_state'], np.nan)

In [ ]:
test['indcount'] = test.groupby('Industry')['Industry'].transform('count')

In [ ]:
test['industry'] = np.where(test['indcount'] >= 4, test['Industry'], np.nan)

In [ ]:
test.replace(to_replace={-1: np.NAN,'-1':np.NAN},inplace=True) 

In [ ]:
test['industry'].value_counts().head(60)

Information Technology Support Services    128
Internet & Web Services                     87
Enterprise Software & Network Solutions     83
Health Care Services & Hospitals            83
Computer Hardware Development               68
Business Consulting                         40
Banking & Lending                           33
Software Development                        32
Biotech & Pharmaceuticals                   28
Insurance Carriers                          26
Investment & Asset Management               22
Advertising & Public Relations              22
Energy & Utilities                          21
Financial Transaction Processing            18
Consumer Product Manufacturing              18
Aerospace & Defense                         15
Colleges & Universities                     12
Film Production                             11
HR Consulting                               11
Food & Beverage Manufacturing               10
Civic & Social Services                      9
Transportatio

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1217 entries, 0 to 1216
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          1217 non-null   object 
 1   Salary Estimate    1217 non-null   object 
 2   Job Description    1217 non-null   object 
 3   Rating             1041 non-null   float64
 4   Company Name       1217 non-null   object 
 5   Location           1217 non-null   object 
 6   Size               1052 non-null   object 
 7   Founded            869 non-null    float64
 8   Type of ownership  1126 non-null   object 
 9   Industry           978 non-null    object 
 10  Sector             978 non-null    object 
 11  Revenue            680 non-null    object 
 12  hourly             1217 non-null   int64  
 13  employer_provided  1217 non-null   int64  
 14  min_salary         1217 non-null   int64  
 15  max_salary         1217 non-null   int64  
 16  avg_salary         1217 

In [ ]:
test = test.drop(columns=['indcount','statecount','comp_count','job_state','company_name','Industry'],axis = 1)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1217 entries, 0 to 1216
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          1217 non-null   object 
 1   Salary Estimate    1217 non-null   object 
 2   Job Description    1217 non-null   object 
 3   Rating             1041 non-null   float64
 4   Company Name       1217 non-null   object 
 5   Location           1217 non-null   object 
 6   Size               1052 non-null   object 
 7   Founded            869 non-null    float64
 8   Type of ownership  1126 non-null   object 
 9   Sector             978 non-null    object 
 10  Revenue            680 non-null    object 
 11  hourly             1217 non-null   int64  
 12  employer_provided  1217 non-null   int64  
 13  min_salary         1217 non-null   int64  
 14  max_salary         1217 non-null   int64  
 15  avg_salary         1217 non-null   float64
 16  job_title          1217 

#### a. onehot, simpleimputer, stdscaler

In [20]:
# I had to do onehotencoding before splitting the data, because we have too many categories, it will raise error
# when transforming the test set

df_dummy = pd.get_dummies(df_model)
df_dummy['avg_salaries_cat'] = pd.cut(df_dummy['avg_salary'], bins=[0.,75,100,135,170,np.inf],
                                      labels=[0,1,2,3,4])
split = StratifiedShuffleSplit(n_splits =1, test_size =0.2, random_state=42)
for train_index, test_index in split.split(df_dummy,df_dummy['avg_salaries_cat']):
    strat_train_set_dum = df_dummy.loc[train_index]
    strat_test_set_dum = df_dummy.loc[test_index]

# drop the avg_salaries_cat so the data is back to its original state
for strat_set in (strat_train_set_dum,strat_test_set_dum,df_dummy):
    strat_set.drop("avg_salaries_cat",axis = 1, inplace=True)

In [22]:
df_dummy

,avg_salary,Rating,age,tools,techs,education,desc_len,hourly,employer_provided,Size_-1,...,"company_name_eTek IT Services, Inc.",company_name_eimagine,"company_name_fairlife, LLC",company_name_gointellects,company_name_iSpot.tv,company_name_inAssist,company_name_kea,company_name_keasis Inc,company_name_shaped.ai Inc.,company_name_zettalogix.Inc
0,187.5,3.7,10,3,2,2,3144,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,136.0,4.3,16,3,6,0,8120,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,100.0,-1.0,-1,4,2,0,1338,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,98.0,4.2,217,7,7,1,5372,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,117.0,4.6,-1,3,0,0,828,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212,121.0,3.9,94,5,5,2,6439,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1213,167.0,3.9,12,3,5,1,3769,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1214,159.0,3.9,100,2,3,2,2988,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1215,221.5,3.2,12,2,4,0,7039,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
x_train_dum = strat_train_set_dum.drop("avg_salary",axis=1)
y_train_dum = strat_train_set_dum['avg_salary'].copy()

num_attribs_dum = list(num_cols.columns)
cat_attribs_dum = list(x_train_dum.iloc[:,8:])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ('imputer',SimpleImputer(missing_values=[np.NaN,-1,'-1'],strategy="mean")),
    ('min_max_scaler', MinMaxScaler()),
])

# full_pipeline = ColumnTransformer([
#     ("num", num_pipeline , num_attribs),
#     ("cat", OneHotEncoder(handle_unknown = "ignore"), cat_attribs),
# ])

model_prepared = num_pipeline.fit_transform(x_train_dum)

In [26]:
model_prepared

array([[0.86666667, 0.21907216, 0.2       , ..., 0.        , 0.        ,
        0.        ],
       [0.76666667, 0.        , 0.2       , ..., 0.        , 0.        ,
        0.        ],
       [0.91666667, 0.        , 0.1       , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.55      , 0.        , 0.1       , ..., 0.        , 0.        ,
        0.        ],
       [0.83333333, 0.29639175, 0.1       , ..., 0.        , 0.        ,
        0.        ],
       [0.83333333, 0.06958763, 0.3       , ..., 0.        , 0.        ,
        0.        ]])

### Imputation using KNN neighbors + feature scaling

In [19]:
cat_cols_copy = cat_cols.copy()
num_cols_copy = num_cols.copy()

In [20]:
cat_cols_copy.replace(to_replace=['-1'],value=np.NAN,inplace=True)

In [21]:
num_cols_copy.replace(to_replace=[-1],value=np.NAN,inplace=True)

In [22]:
cat_cols_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Size               847 non-null    object
 1   Type of ownership  907 non-null    object
 2   Industry           794 non-null    object
 3   Sector             794 non-null    object
 4   Revenue            549 non-null    object
 5   job_title          973 non-null    object
 6   job_state          973 non-null    object
 7   seniority          973 non-null    object
 8   company_name       973 non-null    object
dtypes: object(9)
memory usage: 68.5+ KB


In [23]:
num_cols_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rating             843 non-null    float64
 1   age                706 non-null    float64
 2   tools              973 non-null    int64  
 3   techs              973 non-null    int64  
 4   education          973 non-null    int64  
 5   desc_len           973 non-null    int64  
 6   hourly             973 non-null    int64  
 7   employer_provided  973 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 60.9 KB


In [36]:
# To perform KNN imputation, we need to normalize the input data and perform One Hot Encoding to categorical variables
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
cat_dummies = pd.get_dummies(cat_cols_copy, drop_first = True)
df_copy= pd.concat([num_cols_copy, cat_dummies], axis = 1)

scaler = MinMaxScaler()
# scaler = StandardScaler()
minmaxscaler = pd.DataFrame(scaler.fit_transform(df_copy), columns = df_copy.columns)

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 3)
knn_imputed_df = pd.DataFrame(imputer.fit_transform(minmaxscaler),columns = df_copy.columns)

In [37]:
knn_imputed_df

,Rating,age,tools,techs,education,desc_len,hourly,employer_provided,Size_10000+ Employees,Size_1001 to 5000 Employees,...,"company_name_eTek IT Services, Inc.",company_name_eimagine,"company_name_fairlife, LLC",company_name_gointellects,company_name_iSpot.tv,company_name_inAssist,company_name_kea,company_name_keasis Inc,company_name_shaped.ai Inc.,company_name_zettalogix.Inc
0,0.800,0.215026,0.2,0.222222,0.000000,0.062626,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.650,0.050086,0.2,0.333333,0.666667,0.380393,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.875,0.044905,0.1,0.000000,0.000000,0.064559,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.775,0.139896,0.2,0.444444,0.000000,0.192825,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.725,0.041451,0.5,0.444444,0.000000,0.215556,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,0.800,0.116580,0.1,0.111111,0.000000,0.080795,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
969,0.700,0.300518,0.1,0.000000,0.000000,0.014226,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
970,0.325,0.028497,0.1,0.111111,0.000000,0.044688,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
971,0.750,0.292746,0.1,0.111111,0.000000,0.059146,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### we will perform imputation by keeping the percentage of each category in the population constant even after imputation

In [164]:
cat_cols_test=cat_cols.copy()

In [165]:
#Revenue Column
cat_cols_test['Revenue'].replace(to_replace=['-1'],value=np.NAN,inplace=True)

In [166]:
test = pd.DataFrame(cat_cols_test.Revenue.values.tolist()).stack().value_counts()

In [167]:
cat_cols_test = replace_cat_list(cat_cols_test,'Revenue',test.index)

In [168]:
# type of Ownership Column
cat_cols_test['Type of ownership'].replace(to_replace=['-1'],value=np.NAN,inplace=True)

In [169]:
test = pd.DataFrame(cat_cols_test['Type of ownership'].values.tolist()).stack().value_counts()

In [170]:
cat_cols_test = replace_cat_list(cat_cols_test,'Type of ownership',test.index)

In [171]:
# Industry Column
cat_cols_test['Industry'].replace(to_replace=['-1'],value=np.NAN,inplace=True)

In [172]:
test = pd.DataFrame(cat_cols_test.Industry.values.tolist()).stack().value_counts()

In [173]:
cat_cols_test = replace_cat_list(cat_cols_test,'Industry',test.index)

In [174]:
# Sector column
cat_cols_test['Sector'].replace(to_replace=['-1'],value=np.NAN,inplace=True)

In [175]:
test = pd.DataFrame(cat_cols_test.Sector.values.tolist()).stack().value_counts()

In [176]:
cat_cols_test = replace_cat_list(cat_cols_test,'Sector',test.index)

In [177]:
# Size Column
cat_cols_test['Size'].replace(to_replace=['-1'],value=np.NAN,inplace=True)

In [178]:
test = pd.DataFrame(cat_cols_test.Size.values.tolist()).stack().value_counts()

In [179]:
cat_cols_test = replace_cat_list(cat_cols_test,'Size',test.index)

### we will perform imputation by keeping the percentage of each category in the population constant even after imputation

In [154]:
def replace_cat_list(df,col,cat_list):
    count_cat_dict_initial = {'Total_cat':0}
    for cat in cat_list:
        count_cat_dict_initial[cat] = df.loc[df[col]==cat,col].count()
        count_cat_dict_initial['Total_cat'] = count_cat_dict_initial.get('Total_cat') + count_cat_dict_initial[cat]
    count_cat_dict_initial['Total'] = len(df[col])
    count_cat_dict_final = {'Total_cat':0}
    for cat in cat_list[:-1]:
        count_cat_dict_final[cat] = math.ceil((count_cat_dict_initial.get(cat)/count_cat_dict_initial.get('Total_cat'))*count_cat_dict_initial.get('Total'))
        count_cat_dict_final['Total_cat'] = count_cat_dict_final.get('Total_cat') + count_cat_dict_final[cat]
    count_cat_dict_final[cat_list[-1]] = count_cat_dict_initial['Total'] - count_cat_dict_final['Total_cat']
    fill_dict = {}
    for cat in cat_list:
        fill_dict[cat] = count_cat_dict_final[cat] - count_cat_dict_initial[cat]
    for cat in cat_list[:]:
        for i in range(fill_dict.get(cat)):
            null_index= list(df.loc[pd.isna(df[col]),:].index)
            if len(null_index) != 0:
                df.loc[null_index.pop(0),col] = cat
            else:
                 break
    return df

In [155]:
num_cols_test = num_cols.copy()

In [156]:
num_cols_test.replace(to_replace=[-1],value=np.NAN,inplace=True)

In [157]:
test = pd.DataFrame(num_cols_test.Rating.values.tolist()).stack().value_counts()

In [158]:
test.index

Float64Index([3.9, 3.7, 4.0, 3.8, 4.2, 4.1, 4.3, 5.0, 3.6, 4.4, 3.5, 3.4, 4.5,
              3.3, 4.6, 3.2, 4.9, 3.1, 4.7, 3.0, 2.9, 4.8, 2.6, 2.7, 2.8, 2.0,
              2.4, 2.3, 1.0, 2.5],
             dtype='float64')

In [159]:
num_cols_test = replace_cat_list(num_cols_test,'Rating',test.index)

In [160]:
test = pd.DataFrame(num_cols_test.age.values.tolist()).stack().value_counts()

In [161]:
test.index

Float64Index([  8.0,   7.0,  19.0,   6.0,  25.0,  17.0,  24.0,  12.0,  11.0,
               20.0,
              ...
              212.0, 147.0,  83.0, 120.0, 104.0, 162.0, 151.0, 187.0, 322.0,
              159.0],
             dtype='float64', length=134)

In [162]:
num_cols_test = replace_cat_list(num_cols_test,'age',test.index)

In [163]:
num_cols_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rating             973 non-null    float64
 1   age                973 non-null    float64
 2   tools              973 non-null    int64  
 3   techs              973 non-null    int64  
 4   education          973 non-null    int64  
 5   desc_len           973 non-null    int64  
 6   hourly             973 non-null    int64  
 7   employer_provided  973 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 60.9 KB


### perform imputation using SimpleImputer for numerical columns

In [113]:
import numpy as np

# Importing the SimpleImputer class
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = -1,strategy ='mean')

imputer = imputer.fit(num_cols)
X = imputer.transform(num_cols)

In [114]:
imputer.statistics_

array([3.93534994e+00, 4.10552408e+01, 2.68139774e+00, 2.81397739e+00,
       6.16649538e-01, 3.06520966e+03, 1.61356629e-01, 6.48509764e-01])

In [115]:
num_cols_imputed= pd.DataFrame(X,columns=num_cols.columns,index=num_cols.index)

## using median instead of mean in a soft voting 

In [ ]:
import numpy as np
from sklearn.ensemble import VotingRegressor

# Define a custom function that returns the median along axis 1
def median_voting(estimators, X):
    # Get the predicted probabilities or scores from each estimator
    predictions = np.asarray([est.predict(X) for est in estimators])
    # Return the median along axis 1
    return np.median(predictions, axis=0)

# Define the base regressors
reg1 = ...
reg2 = ...
reg3 = ...

# Define the soft voting regressor with median voting
voting_reg = VotingRegressor(estimators=[("reg1", reg1), ("reg2", reg2), ("reg3", reg3)])
# Set the voting method to the custom function
voting_reg.voting = median_voting

# Fit and predict
voting_reg.fit(X_train, y_train)
y_pred = voting_reg.predict(X_test)

## use a Pipeline for each model, where you apply the transformation and then the model in sequence. This way, you can avoid data leakage and ensure that each model gets the appropriate input. You can then use a VotingClassifier to combine the predictions from each Pipeline.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso

# Define the transformations for each model
svm_transform = ... # some transformation for SVM
rf_transform = ... # some transformation for Random Forest
lasso_transform = ... # some transformation for Lasso

# Define the models
svm_model = SVC()
rf_model = RandomForestClassifier()
lasso_model = Lasso()

# Define the pipelines
svm_pipeline = Pipeline([("transform", svm_transform), ("model", svm_model)])
rf_pipeline = Pipeline([("transform", rf_transform), ("model", rf_model)])
lasso_pipeline = Pipeline([("transform", lasso_transform), ("model", lasso_model)])

# Define the voting classifier
voting_clf = VotingClassifier(estimators=[("svm", svm_pipeline), ("rf", rf_pipeline), ("lasso", lasso_pipeline)])

# Fit and predict
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)